In [1]:
from stable_baselines3.common.env_checker import check_env
from gym_reversi import ReversiEnv

env = ReversiEnv(player_color='black', opponent = "random", observation_type="numpy3c", 
                illegal_place_mode="lose", board_size=8)
# It will check your custom environment and output additional warnings if needed
check_env(env)


D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:30: UserWarning: It seems that your observation  is an image but its `dtype` is (int32) whereas it has to be `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  warnings.warn(
D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:38: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:51: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom features extractor cf. https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  warnings.warn(


In [ ]:
# import sys
# import os
# current_path = os.getcwd()
# sys.path.append(current_path)

from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import PPO
import time
from gym_reversi import ReversiEnv

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
# vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
# env = make_atari_env("BreakoutNoFrameskip-v4", seed=0)
# vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=4, 
# #                          seed=0
#                         )
# # Frame-stacking with 4 frames
# vec_env = VecFrameStack(vec_env, n_stack=4)

env = ReversiEnv(opponent="random", is_train=True, board_size=4, 
                 replace_invalid_action=False, only_game_finished_reward=True,verbose=0)


model = PPO('MlpPolicy', env,
              policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=2.5e-4,
              ent_coef=0.01,
              n_steps=128,  # n_steps=128, 
              n_epochs=4,
              batch_size=256,  # batch_size=256,
              gamma=0.99,
              gae_lambda=0.95,
              clip_range=0.1,
              vf_coef=0.5,
              verbose=1,
              tensorboard_log="models/Reversi_ppo/")

# model = PPO.load("models/Reversi_ppo/model", env=env)

# model.learn(int(2e4))
model.learn(total_timesteps=100_0000)
model.save("models/Reversi_ppo/model")

print(f"train time: {time.time()-t0}")

# vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1)
# vec_env = VecFrameStack(vec_env, n_stack=4)

# model = PPO.load("models/Reversi_ppo/model", env=env)
# obs, info = env.reset()
# while True:
#     action, _states = model.predict(obs, deterministic=False)
#     print(f"action: {action}")
#     obs, rewards, dones, truncated, info = env.step(action)
#     env.render("human")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to models/Reversi_ppo/PPO_11


D:\apps\Anaconda3\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 128`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 128
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=128 and n_envs=1)
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.2      |
|    ep_rew_mean     | -1       |
| time/              |          |
|    fps             | 248      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 128      |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.28          |
|    ep_rew_mean          | -1            |
| time/                   |               |
|    fps                  | 232           |
|    iterations           | 2             |
|    time_elapsed         | 1             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | 0.00060998555 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | -2.77         |
|    explained_variance   | -171          |


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.31          |
|    ep_rew_mean          | -1            |
| time/                   |               |
|    fps                  | 256           |
|    iterations           | 11            |
|    time_elapsed         | 5             |
|    total_timesteps      | 1408          |
| train/                  |               |
|    approx_kl            | 0.00029427372 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | -2.75         |
|    explained_variance   | -124          |
|    learning_rate        | 0.00025       |
|    loss                 | -0.0341       |
|    n_updates            | 40            |
|    policy_gradient_loss | -0.0043       |
|    value_loss           | 0.00534       |
-------------------------------------------
------------------------------------------
| rollout/                |      

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.31          |
|    ep_rew_mean          | -1            |
| time/                   |               |
|    fps                  | 258           |
|    iterations           | 20            |
|    time_elapsed         | 9             |
|    total_timesteps      | 2560          |
| train/                  |               |
|    approx_kl            | 0.00053627556 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | -2.73         |
|    explained_variance   | -32.8         |
|    learning_rate        | 0.00025       |
|    loss                 | -0.0351       |
|    n_updates            | 76            |
|    policy_gradient_loss | -0.00387      |
|    value_loss           | 0.00213       |
-------------------------------------------
------------------------------------------
| rollout/                |      

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.36         |
|    ep_rew_mean          | -0.07        |
| time/                   |              |
|    fps                  | 282          |
|    iterations           | 667          |
|    time_elapsed         | 302          |
|    total_timesteps      | 85376        |
| train/                  |              |
|    approx_kl            | 0.0012765871 |
|    clip_fraction        | 0.0234       |
|    clip_range           | 0.1          |
|    entropy_loss         | -0.319       |
|    explained_variance   | 0.52         |
|    learning_rate        | 0.00025      |
|    loss                 | 0.184        |
|    n_updates            | 2664         |
|    policy_gradient_loss | -0.00249     |
|    value_loss           | 0.387        |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.62         |
|    ep_rew_mean          | -0.02        |
| time/                   |              |
|    fps                  | 282          |
|    iterations           | 677          |
|    time_elapsed         | 307          |
|    total_timesteps      | 86656        |
| train/                  |              |
|    approx_kl            | 0.0023135673 |
|    clip_fraction        | 0.0156       |
|    clip_range           | 0.1          |
|    entropy_loss         | -0.272       |
|    explained_variance   | 0.657        |
|    learning_rate        | 0.00025      |
|    loss                 | 0.127        |
|    n_updates            | 2704         |
|    policy_gradient_loss | -0.00273     |
|    value_loss           | 0.279        |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.32         |
|    ep_rew_mean          | -0.15        |
| time/                   |              |
|    fps                  | 282          |
|    iterations           | 687          |
|    time_elapsed         | 311          |
|    total_timesteps      | 87936        |
| train/                  |              |
|    approx_kl            | 0.0019744574 |
|    clip_fraction        | 0.0371       |
|    clip_range           | 0.1          |
|    entropy_loss         | -0.288       |
|    explained_variance   | 0.615        |
|    learning_rate        | 0.00025      |
|    loss                 | 0.153        |
|    n_updates            | 2744         |
|    policy_gradient_loss | -0.00567     |
|    value_loss           | 0.35         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.04         |
|    ep_rew_mean          | -0.28        |
| time/                   |              |
|    fps                  | 282          |
|    iterations           | 697          |
|    time_elapsed         | 316          |
|    total_timesteps      | 89216        |
| train/                  |              |
|    approx_kl            | 0.0020846291 |
|    clip_fraction        | 0.0254       |
|    clip_range           | 0.1          |
|    entropy_loss         | -0.253       |
|    explained_variance   | 0.646        |
|    learning_rate        | 0.00025      |
|    loss                 | 0.148        |
|    n_updates            | 2784         |
|    policy_gradient_loss | -0.00174     |
|    value_loss           | 0.313        |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.53         |
|    ep_rew_mean          | 0.04         |
| time/                   |              |
|    fps                  | 282          |
|    iterations           | 707          |
|    time_elapsed         | 320          |
|    total_timesteps      | 90496        |
| train/                  |              |
|    approx_kl            | 0.0011262922 |
|    clip_fraction        | 0.0254       |
|    clip_range           | 0.1          |
|    entropy_loss         | -0.252       |
|    explained_variance   | 0.731        |
|    learning_rate        | 0.00025      |
|    loss                 | 0.103        |
|    n_updates            | 2824         |
|    policy_gradient_loss | -0.00259     |
|    value_loss           | 0.234        |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

In [3]:
tensorboard --logdir ./reversi/models/Reversi_ppo/PPO_7/ --port=6016

8

In [4]:
25000/3600

6.944444444444445

In [5]:
import time
from stable_baselines3 import PPO
from gym_reversi import ReversiEnv

env = ReversiEnv(opponent="random", board_size=4, player_color='black', 
                 replace_invalid_action=True, only_game_finished_reward=True, verbose=0)

model = PPO.load("models/Reversi_ppo/model", env=env)

max_round = 1000

total_round = 0
total_win = 0
total_failure = 0
total_equal = 0


t0=time.time()
obs, info = env.reset()
while total_round < max_round:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, truncated, info = env.step(action)

#     print(f"---- round:{total_round} --------")
#     print(f"action: {action}")
#     env.render("human")

    if dones:
        print(f"\n\n---- round:{total_round} --------")
#         env.render("human")
        obs, info = env.reset()
        total_round += 1
        if rewards > 0:
            total_win+=1
        elif rewards < 0:
            total_failure += 1
        else:
            total_equal += 1

        print(f"total_win:{total_win}, total_failure: {total_failure}, total_equal:{total_equal}\n\n")

# print(f"total_win:{total_win}, total_failure: {total_failure}")
print(f"train time: {time.time()-t0}")


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
action: 44
       1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  | 
 -------------------------------------------------------
 1  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |
 -------------------------------------------------------
 2  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |
 -------------------------------------------------------
 3  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |
 -------------------------------------------------------
 4  |  O  |  O  |  O  |  W  |  B  |  O  |  O  |  O  |
 -------------------------------------------------------
 5  |  O  |  O  |  O  |  B  |  W  |  O  |  O  |  O  |
 -------------------------------------------------------
 6  |  O  |  O  |  O  |  O  |  B  |  W  |  O  |  O  |
 -------------------------------------------------------
 7  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |
 -------------------------------------------------------
 8  |  O  |  O  |  O  |

 2  |  O  |  B  |  O  |  W  |  W  |  B  |  O  |  O  |
 -------------------------------------------------------
 3  |  W  |  W  |  W  |  W  |  W  |  B  |  W  |  O  |
 -------------------------------------------------------
 4  |  O  |  W  |  W  |  B  |  B  |  W  |  W  |  O  |
 -------------------------------------------------------
 5  |  W  |  W  |  B  |  W  |  B  |  B  |  W  |  O  |
 -------------------------------------------------------
 6  |  W  |  W  |  W  |  W  |  B  |  W  |  W  |  O  |
 -------------------------------------------------------
 7  |  W  |  W  |  B  |  B  |  W  |  W  |  W  |  B  |
 -------------------------------------------------------
 8  |  W  |  O  |  B  |  W  |  W  |  W  |  W  |  O  |
 -------------------------------------------------------
action: 52
       1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  | 
 -------------------------------------------------------
 1  |  O  |  O  |  O  |  O  |  W  |  O  |  O  |  O  |
 ---------------------------------------------

1

In [3]:
from gymnasium.utils import seeding


In [4]:
seeding.np_random??

In [5]:
import numpy.random as npr
rng=npr.default_rng()
size=(3,4)
C=rng.uniform(4,7,size)
print(f"{C=}")


C=array([[5.43019184, 6.40250697, 6.1004084 , 5.13867582],
       [6.29431658, 5.80315648, 5.16903573, 4.87083518],
       [4.42687125, 4.45198253, 5.73728282, 6.59846583]])


In [10]:
rng.integers(2)

0

In [ ]:
np.random.Generator()